# Install Required Packages

In [ ]:
pip install -r requirements.txt

# Create Database and Insert Records

In [ ]:
import sqlite3

db_name = "users.db"

# Some records
users = [
    ("Alice", "alice@helloworld.com", "2024-01-15"),
    ("Bob", "bob@helloworld.com", "2024-02-20"),
    ("Carol", "carol@helloworld.com", "2024-03-10"),
    ("Dave", "dave@helloworld.com", "2024-04-05"),
    ("Eve", "eve@helloworld.com", "2025-12-12"),
    ("Frank", "frank@helloworld.com", "2025-12-18"),
    ("Grace", "grace@helloworld.com", "2024-05-02"),
    ("Hank", "hank@helloworld.com", "2024-05-10"),
    ("Ivy", "ivy@helloworld.com", "2025-12-15"),
    ("Jack", "jack@helloworld.com", "2024-05-20")
]

# Connect to db
conn = sqlite3.connect(db_name)
cursor = conn.cursor()

# Create users table
cursor.execute("""
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        email TEXT UNIQUE NOT NULL,
        signup_date TEXT
    );
""")

# Insert the data
cursor.executemany("""
    INSERT INTO users (name, email, signup_date)
    VALUES (?, ?, ?);
""", users)


print("Table created and data inserted.")

# Commit changes to db and clean up
conn.commit()
cursor.close()
conn.close()

# Create Function to Query Db to Get User Signups By Month/Year

In [ ]:
def get_user_signups_by_month_year(month: int, year: int) -> int:
    """
    Return the number of users who signed up in the given month and year.

    Args:
        month (int): Month number (1-12)
        year (int): Full year (e.g., 2025)

    Returns:
        int: Count of users signed up in that month and year.
    """
    if not (1 <= month <= 12):
        raise ValueError("Month must be between 1 and 12")

    # Format month and year
    month_str = f"{month:02d}"
    year_str = str(year)

    # Construct date range
    start_date = f"{year_str}-{month_str}-01"
    end_date = f"{year_str}-{month_str}-31"  

    query = """
        SELECT COUNT(*) FROM users WHERE signup_date BETWEEN ? AND ?;
    """

    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    cursor.execute(query, (start_date, end_date))
    count = cursor.fetchone()[0]

    cursor.close()
    conn.close()

    return count

# Define Function Metadata for GPT Tool Use

In [ ]:
import openai
import json

openai.api_key = "<replace-with-your-api-key>"

tool = {
    "type": "function",
    "function": {
        "name": "get_user_signups_by_month_year",
        "description": "Return the number of users who signed up in the given month and year.",
        "parameters": {
            "type": "object",
            "properties": {
                "month": {
                    "type": "integer",
                    "description": "The month number (1-12)"
                },
                "year": {
                    "type": "integer",
                    "description": "The 4-digit year"
                }
            },
            "required": ["month", "year"]
        }
    }
}

# Using the Tool

In [ ]:
def llm_prompt(prompt, model="gpt-4o"):

    # Prepare initial user message for the model
    messages = [{"role": "user", "content": prompt}]
    
    # Call the language model with optional tool usage
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        tools=[tool],
        tool_choice="auto",
        temperature=0.7
    )

    # Get the model's first response message
    print("response", response)
    message = response.choices[0].message
    print(message)

    # Check if the model requested a tool call
    tool_call = message.tool_calls[0] if message.tool_calls else None
    print(tool_call)

    # If no tool call, return the model's direct response
    if not tool_call:
        return message.content

    # Parse tool arguments and select the appropriate function to call
    args = json.loads(tool_call.function.arguments)
    function_map = {
        "get_user_signups_by_month_year": get_user_signups_by_month_year
    }

    # Get function from map and validate
    func = function_map.get(tool_call.function.name)
    if not func:
        return f"Unknown tool: {tool_call.function.name}"

    # Call the function with args
    result = func(**args)

    # Append tool call result to message history
    messages += [
        message.model_dump(),
        {
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": tool_call.function.name,
            "content": str(result)
        }
    ]

    # Call the model again with updated context to generate a follow-up response
    followup = openai.chat.completions.create(
        model=model,
        messages=messages
    )

    # Return the content of the follow-up message
    return followup.choices[0].message.content

In [ ]:

# --- Example usage ---
response = llm_prompt("How many users signed up in the month of December 2025?")
print(response)